# LSTM with no 0 class and holdig out 2 subjects

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from keras import losses
from keras.models import Sequential
from keras.layers import (Dense, Embedding, Reshape, Activation, 
                          SimpleRNN, LSTM, Convolution1D, 
                          MaxPooling1D, Dropout, Bidirectional)
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
col = []
for ea_ideal in range(1,18):
    exec(f"df_subj_ideal_{ea_ideal} = pd.read_csv('/content/drive/My Drive/realistic_sensor_displacement/subject{ea_ideal}_ideal.log', sep='\t', names = range(120))")
    col.append(ea_ideal)

In [ ]:
col

In [ ]:
df_subj_ideal_1.shape

In [ ]:
subj_num_ls = [df_subj_ideal_1,df_subj_ideal_2,df_subj_ideal_3,df_subj_ideal_4,df_subj_ideal_5,
              df_subj_ideal_6,df_subj_ideal_7,df_subj_ideal_8,df_subj_ideal_9,df_subj_ideal_10,
              df_subj_ideal_11,df_subj_ideal_12,df_subj_ideal_13,df_subj_ideal_14,df_subj_ideal_15,
              df_subj_ideal_16,df_subj_ideal_17]

In [ ]:
ls_of_dfs = []
for num in range(1,18):
    for activity in subj_num_ls[num-1][119].unique():
        ls_of_dfs.append(f"df_idealSubj{num}_{activity}")

In [ ]:
ls_of_dfs[:5]

## Without 0 class and 2 subjects

In [ ]:
ls_of_dfs_noZero = []
for num in range(1,18):
    for activity in subj_num_ls[num-1][119].unique():
#         if num == 2 or num == 3:    #skip subject 
#             continue
        if activity == 0:    #skip 0 activity (no activity labeled)
            continue
        ls_of_dfs_noZero.append(f"df_noZeroClass_idealSubj{num}_{activity}")

In [ ]:
ls_of_dfs_noZero[30:40]

In [ ]:
for num in range(1,18):
    for activity in subj_num_ls[num-1][119].unique():
        if activity == 0:
            continue
        exec(f"df_noZeroClass_idealSubj{num}_{activity} = df_subj_ideal_{num}[df_subj_ideal_{num}[119]=={activity}]")

In [ ]:
len(ls_of_dfs_noZero)

In [ ]:
df_noZeroClass_idealSubj9_30.head(2)

In [ ]:
ls_of_dfs_noZero[-1]

In [ ]:
col = []

timeSteps = 100

for individual_df in ls_of_dfs_noZero:
    for i in range((len(vars()[individual_df]) + timeSteps - 1) // timeSteps ) :
        col.append(vars()[individual_df][i * timeSteps:(i + 1) * timeSteps].to_numpy())

In [ ]:
ls_timeSteps_master_noZero = [arr for arr in col if arr.shape == (100, 120)]

In [ ]:
len(ls_timeSteps_master_noZero)

In [ ]:
ls_timeSteps_master_noZero[6000][0,-1]

In [ ]:
timeStepsStack_noZero = []
timeStepsStack_noZero = np.stack(ls_timeSteps_master_noZero)

In [ ]:
timeStepsStack_noZero.shape

In [ ]:
y_noZero = timeStepsStack_noZero[:,:,-1]
X_noZero = timeStepsStack_noZero[:,:,2:-1]

In [ ]:
y_noZero = y_noZero[:,0]

In [ ]:
plt.hist(y_noZero)

In [ ]:
print(y_noZero.shape, X_noZero.shape)

## Test/Train Split without 0 class

In [ ]:
X_train_noZero, X_test_noZero, y_train_noZero, y_test_noZero = train_test_split(X_noZero, y_noZero, test_size=.2, random_state=17)

In [ ]:
print(X_train_noZero.shape, X_test_noZero.shape)

In [ ]:
print(y_train_noZero.shape, y_test_noZero.shape)

## LSTM

In [ ]:
# design network
model = Sequential()
model.add(LSTM(50, input_shape=(X_train_noZero.shape[1],X_train_noZero.shape[2])))
model.add(Dense(len(np.unique(y_train_noZero))))
model.add(Activation('softmax'))
model.compile(loss=losses.sparse_categorical_crossentropy, 
              optimizer='adam', 
              metrics=['accuracy'])

model.summary()

In [ ]:
# fit network
print('Train...')
model.fit(X_train_noZero, y_train_noZero, epochs=100, 
          validation_split=.25, 
          callbacks=[
        keras.callbacks.EarlyStopping(patience=8, verbose=1, restore_best_weights=True),
        keras.callbacks.ReduceLROnPlateau(factor=.5, patience=3, verbose=1)]
         )

# evaluate the model
#loss, accuracy, f1_score, precision, recall = model.evaluate(X_test_withZero, y_test_withZero, batch_size=X_test_withZero.shape[0])
score, acc = model.evaluate(X_test_noZero, y_test_noZero, 
                            #batch_size = X_test_noZero.shape[0]
                           )
print('Test score:', score)
print('Test accuracy:', acc)